In [ ]:
import torch
from mpmath import barnesg
from torch import nn, einsum, broadcast_tensors
import torch.nn.functional as F

from torch_geometric.datasets import QM9

In [ ]:
dataset1 = QM9(root = '.')

In [ ]:
dataset1[0]

In [ ]:
train_y = dataset1.y[:,4]

In [ ]:
from rdkit import Chem, RDLogger

In [ ]:
one_hot(5), atomic_number(1), aromatic(1), sp(1), sp2(1), sp3(1), num_hs(1)

In [ ]:
RDLogger.DisableLog('rdApp.*')

In [ ]:
dataset1["A"]

In [1]:
from src.dataset import *

C:\Users\alwas\Desktop\egnn_tda\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = QM9Dataset(root=".", target_y="gap", max_atoms=10, force_reload=False)
print(dataset[0].node_attr.shape, dataset[0].y)

Processing...
100%|██████████| 133885/133885 [01:25<00:00, 1573.57it/s]
Done!


torch.Size([5, 6]) tensor([13.7363])


In [16]:
dataset[0]

Data(x=[5, 11], edge_index=[2, 8], edge_attr=[8, 4], y=[1], pos=[5, 3], z=[5], smiles='[H]C([H])([H])[H]', name='gdb_1', idx=[1], node_attr=[5, 6])

In [3]:
dataset[0].node_attr

tensor([[1., 0., 0., 0., 6., 4.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.]])

In [3]:
from src.gnn import *

In [ ]:
dataset[0].node_attr

In [ ]:
dataset[0].edge_attr

In [5]:
model = EquivariantGNN(input_node_dim = 6 + 3,
                       input_edge_dim = 4,
                       output_dim = 4,
                       attention=False,
                       num_coords=3,
                       num_velocities=0,
                       update_coords=False,
                       hidden_dim=128,
                       num_layers=4,
                       predict_type="node_pooling")

In [6]:
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
import torch

# split
n = len(dataset)
n_train = int(0.90 * n)
n_test = n - n_train
train_ds, test_ds = random_split(dataset, [n_train, n_test], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

In [23]:
for batch in train_loader:
    a = batch

In [28]:
a.batch

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,
         7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11,
        11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13,
        13])

In [92]:
[ d.edge_index for d in dataset]

[tensor([[0, 0, 0, 0, 1, 2, 3, 4],
         [1, 2, 3, 4, 0, 0, 0, 0]]),
 tensor([[0, 0, 0, 1, 2, 3],
         [1, 2, 3, 0, 0, 0]]),
 tensor([[0, 0, 1, 2],
         [1, 2, 0, 0]]),
 tensor([[0, 0, 1, 1, 2, 3],
         [1, 3, 0, 2, 1, 0]]),
 tensor([[0, 0, 1, 2],
         [1, 2, 0, 0]]),
 tensor([[0, 0, 0, 1, 2, 3],
         [1, 2, 3, 0, 0, 0]]),
 tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 3, 4, 5, 6, 7],
         [1, 2, 3, 4, 0, 5, 6, 7, 0, 0, 0, 1, 1, 1]]),
 tensor([[0, 0, 0, 0, 1, 1, 2, 3, 4, 5],
         [1, 2, 3, 4, 0, 5, 0, 0, 0, 1]]),
 tensor([[0, 0, 0, 0, 1, 1, 2, 2, 3, 4, 5, 6],
         [1, 3, 4, 5, 0, 2, 1, 6, 0, 0, 0, 2]]),
 tensor([[0, 0, 0, 0, 1, 1, 2, 3, 4, 5],
         [1, 3, 4, 5, 0, 2, 1, 0, 0, 0]]),
 tensor([[0, 0, 0, 0, 1, 1, 1, 2, 3, 4, 5, 6],
         [1, 3, 4, 5, 0, 2, 6, 1, 0, 0, 0, 1]]),
 tensor([[0, 0, 0, 1, 1, 1, 2, 3, 4, 5],
         [1, 3, 4, 0, 2, 5, 1, 0, 0, 1]]),
 tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 4, 5, 6, 7, 8],
         [1, 3, 4, 5, 0, 2, 6, 7, 1, 8, 0

In [7]:
from tqdm import tqdm
from torch import nn

n_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)

model.to(device)

train_losses = []
loss_fn = nn.MSELoss()

for epoch in tqdm(range(n_epochs)):

    for batch in train_loader:
        batch = batch.to(device)

        node_attr = [torch.cat((batch[i].node_attr.float(), batch[i].pos.float()), dim=1) for i in range(len(batch))]
        edge_index = [batch[i].edge_index for i in range(len(batch))]
        edge_attr = [batch[i].edge_attr for i in range(len(batch))]
        y = batch.y

        optimizer.zero_grad()
        out = model(
            node_features=node_attr,
            edge_indices=edge_index,
            edges_features=edge_attr,
            batch_size=len(batch),
        )
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        train_losses.append(loss)

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: EquivariantGNN.forward() got an unexpected keyword argument 'batch_size'

In [13]:
batch

DataBatch(x=[308, 11], edge_index=[2, 594], edge_attr=[594, 4], y=[32], pos=[308, 3], z=[308], smiles=[32], name=[32], idx=[32], node_attr=[308, 6], batch=[308], ptr=[33])

In [10]:
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 100

node_features = torch.concat((dataset[0].node_attr, dataset[0].pos), dim=1).to(device)
edge_indices = dataset[0].edge_index.to(device)
edges_features = dataset[0].edge_attr.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

model.to(device)

for i in range(epochs):

    optimizer.zero_grad()
    out = model(node_features = node_features,
          edge_indices = edge_indices,
          edges_features = edges_features,
          batch_size=1)[0]

    loss = F.l1_loss(out,  dataset[0].y.to(device))
    loss.backward()
    optimizer.step()


    print(loss)

C:\Users\alwas\AppData\Local\Temp\ipykernel_22900\2970388740.py:24: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(out,  dataset[0].y.to(device))


tensor(13.8030, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(13.1633, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(12.1769, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(10.6496, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(8.0704, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(3.3416, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(5.7466, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(4.2451, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0052, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(3.7667, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(5.2693, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(5.8192, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(5.8207, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(5.4044, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(4.5667, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(3.2144, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1.1313, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(2.1094, device='cuda

In [ ]:
model(node_features = node_features,
          edge_indices = edge_indices,
          edges_features = edges_features,
          batch_size=1)[0]

In [ ]:
dataset[0].y

In [ ]:
device = torch.cuda.device("cuda")

In [ ]:
device

In [1]:
from src.data import *

C:\Users\alwas\Desktop\egnn_tda\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
qm9 = load_qm9(".")
train, val, test = split(qm9, train_ratio=0.8)

Extracting .\raw\qm9.zip
Processing...
100%|██████████| 133885/133885 [01:32<00:00, 1451.61it/s]
Done!


In [8]:
from tqdm import tqdm
from torch import nn

n_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)

model.to(device)

train_losses = []
loss_fn = nn.MSELoss()

for epoch in tqdm(range(n_epochs)):

    for batch in train:
        batch = batch.to(device)

        node_attr = [torch.cat((batch[i].node_attr.float(), batch[i].pos.float()), dim=1) for i in range(len(batch))]
        edge_index = [batch[i].edge_index for i in range(len(batch))]
        edge_attr = [batch[i].edge_attr for i in range(len(batch))]
        y = batch.y

        optimizer.zero_grad()
        out = model(
            node_features=node_attr,
            edge_indices=edge_index,
            edges_features=edge_attr,
            batch_size=len(batch),
        )
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        train_losses.append(loss)

  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'GlobalStorage' object has no attribute 'node_attr'

In [13]:
batch

DataBatch(x=[718, 11], edge_index=[2, 1440], edge_attr=[1440, 4], y=[64], pos=[718, 3], batch=[718], ptr=[65])

In [14]:
from src.model3 import *

In [16]:
model3 = E3GCL(6)

In [18]:
model3(0)

TypeError: E3GCL.forward() missing 2 required positional arguments: 'x' and 'edge_index'